<a href="https://colab.research.google.com/github/jakewalter/intro_seismology/blob/main/final_project_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the necessary libraries and load the dat

In [ ]:
!wget https://jakewalter.mynetgear.com/data/data.zip
!unzip data.zip

!pip install git+https://github.com/gaprieto/multitaper.git
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from multitaper import MTSpec, MTCross

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
#from scipy.signal.windows import pmtm
from scipy.stats import linregress

import numpy as np

def nextpow2(n):
    # Next higher power of 2.
    # Returns the first p such that 2^p >= abs(n).
    # Class support for input N: float, integer

    if not isinstance(n, (int, np.integer)):
        f, p = np.frexp(np.abs(n))

        # Check for exact powers of 2.
        k = (f == 0.5)
        p[k] = p[k] - 1

        # Check for infinities and NaNs
        k = ~np.isfinite(f)
        p[k] = f[k]
    else:  # integer case
        p = np.zeros(np.shape(n), dtype=np.int64)
        nabs = np.abs(n)
        x = np.right_shift(nabs, 1)
        while np.any(x):
            p += np.sign(x)
            x = np.right_shift(x, 1)
        # Adjust for all non powers of 2
        p += np.maximum(0, np.sign(nabs - np.left_shift(np.ones_like(p), p)))

    return p


#!wget https://jakewalter.mynetgear.com/data/data.zip

#!pip install spectrum


import scipy.io as sio
sta = sio.loadmat('station.mat')
# fracout = simpl['fracoutsimple']
# fracouta = simpl['fracoutsimplea']

# Load data
M6_2017 = sio.loadmat('M6_2017.mat')
M5_2016 = sio.loadmat('M5_2016.mat')
#M6_2017 = np.load('M6_2017.npy', allow_pickle=True).item()

# Load pre-existing picks
ppick_exist = sio.loadmat('ppick_exist.mat')
pick = ppick_exist['ppick'][0][1][0]

twin_frompick = [-1, 8]
twin_ampratio = [-0.5, 4]


x = 0
stid1 = [x[0] for x in M5_2016['data']['stid'][0][0][x]]
stid2 = [x[0] for x in M6_2017['data']['stid'][0][0][x]]

if (set(stid1) & set(stid2)):
    stid_common = (set(stid1) & set(stid2))

stid_common = list(stid_common)
stid1_ind = []
for i, j in enumerate(stid1):
    if j in stid_common:
        stid1_ind.append(i)

stid2_ind = []
for i, j in enumerate(stid2):
    if j in stid_common:
        stid2_ind.append(i)

samprate_common = M5_2016['data']['samprate'][0][0][0][stid1_ind]

seis1_common = M5_2016['data']['seis'][0][0][0][stid1_ind]
seis1_common_names = M5_2016['data']['stid'][0][0][0][stid1_ind]
seis2_common = M6_2017['data']['seis'][0][0][0][stid2_ind]
seis2_common_names = M6_2017['data']['stid'][0][0][0][stid2_ind]

print(seis1_common_names,seis2_common_names)

ppick1_common = ppick_exist['ppick'][0][0][0][stid1_ind]
ppick2_common = ppick_exist['ppick'][0][1][0][stid2_ind]

ampratio = np.zeros(len(stid_common))
sampratio = np.zeros(len(stid_common))


Next we compare the common station waveforms and compare the amplitudes in the time domain and examing the energy in the frequency domain. Specifically we calculate an amplitude ratio for each set of observations.

In [ ]:


for istac, stac_id in enumerate(seis1_common_names):
    print(stac_id)
    print(seis1_common_names[istac][0])
    print(seis2_common_names[istac][0])
    tnwin_frompick = np.round(np.array(twin_frompick) * samprate_common[istac]).astype(int)
    tnwin_ampratio = np.round(np.array(twin_ampratio) * samprate_common[istac]).astype(int)
    tn1 = np.round(ppick1_common[istac] * samprate_common[istac]) + tnwin_frompick
    tn2 = np.round(ppick2_common[istac] * samprate_common[istac]) + tnwin_frompick

    #stac_id = list(stid2).index(stac_id)
    trace1a = seis1_common[istac]
    is1 = []
    for i in trace1a:
        is1.append(i[0])
    trace1a = np.array(is1)
    trace1 = trace1a[int(tn1[0]):int(tn1[1])]

    trace2a = seis2_common[istac]
    is2 = []
    for i in trace2a:
        is2.append(i[0])
    trace2a = np.array(is2)
    trace2 = trace2a[int(tn2[0]):int(tn2[1])]
    timestamp = np.arange(tnwin_frompick[0], tnwin_frompick[1]) / samprate_common[istac]

    nlen = tnwin_frompick[1] - tnwin_frompick[0]
    Nfft = 2 ** nextpow2(nlen)
    kspec=6
    Py1 = MTSpec(trace1,3.5,kspec,1/samprate_common[istac])
    freq1 ,spec1   = Py1.rspec()
    kspec=6
    Py2 = MTSpec(trace2,3.5,kspec,1/samprate_common[istac])
    freq2 ,spec2   = Py2.rspec()
    # spec1, freq1 = pmtm(trace1, NW=3.5, NFFT=Nfft, Fs=samprate_common[istac])
    # spec2, freq2 = pmtm(trace2, NW=3.5, NFFT=Nfft, Fs=samprate_common[istac])
    nfg = np.where(freq1 > 0)[0]
    omega = 2 * np.pi * freq1[nfg]
    dspec1 = np.sqrt(spec1[nfg]) / omega
    dspec2 = np.sqrt(spec2[nfg]) / omega
    freq = freq1[nfg]

    XCF = np.correlate(trace1, trace2, mode='full')
    lags = np.arange(-len(trace1) + 1, len(trace1))
    nshift = lags[np.argmax(XCF)]
    tshift = nshift / samprate_common[istac]

    b, _, _, _, _ = linregress(trace1, trace2)
    trace1_filt = filtfilt(*butter(4, [1, 8], 'bandpass', fs=samprate_common[istac]), trace1a)
    trace2_filt = filtfilt(*butter(4, [1, 8], 'bandpass', fs=samprate_common[istac]), trace2a)

    trace1_amp = trace1_filt[int(np.round(ppick1_common[istac]*samprate_common[istac]+tnwin_ampratio[0] + nshift)):int(np.round(ppick1_common[istac]*samprate_common[istac]+tnwin_ampratio[1] + nshift))]
    trace2_amp = trace2_filt[int(np.round(ppick2_common[istac]*samprate_common[istac]+tnwin_ampratio[0])):int(np.round(ppick2_common[istac]*samprate_common[istac]+tnwin_ampratio[1]))]
    b = linregress(trace1_amp, trace2_amp)


    plt.figure(istac)
    plt.subplot(2, 2, 1)
    plt.plot(timestamp - tshift, trace1 * b[0], 'r', linewidth=1.5)
    plt.plot(timestamp, trace2, 'b', linewidth=1.5)
    plt.xlim(-2, 5)
    plt.xlabel('time (seconds)')
    plt.ylabel('amplitude')
    plt.legend(['event in 2016', 'event in 2017'], loc='best')
    plt.title(seis1_common_names[istac][0])
    plt.subplot(2, 2, 2)
    ampratio[istac] = b[0]
    plt.plot(trace1_amp, trace2_amp, 'r.', markersize=15)
    plt.plot(trace1_amp, b[1] + trace1_amp * b[0], 'k', linewidth=1.5)
    plt.xlabel('amplitude:event in 2016')
    plt.ylabel('amplitude:event in 2017')
    plt.title(f'ampratio={b[0]}')



    plt.subplot(2, 2, 3)
    plt.loglog(freq, dspec1, 'r', linewidth=1.5)
    plt.loglog(freq, dspec2, 'b', linewidth=1.5)
    plt.legend(['event in 2016', 'event in 2017'], loc='best')
    plt.xlabel('frequency (Hz)')
    plt.ylabel('spectral amplitude')
    plt.subplot(2, 2, 4)
    specratio = dspec2 / dspec1
    plt.loglog(freq, specratio, 'k', linewidth=1.5)
    plt.xlabel('frequency (Hz)')
    plt.ylabel('spectral ratio')
    #plt.ylim([min(specratio), max(specratio)])
    sampratio[istac] = np.mean(specratio[(freq >= 0.3) & (freq <= 0.8)])
    plt.plot([0.3, 0.8], [sampratio[istac], sampratio[istac]], 'r-', linewidth=1)
    plt.title(f'spec ampratio = {sampratio[istac]}')

# Now examine the amplitude ratio and convert to magnitude
mb, mw = 6.3, 6.3
mb_2016 = mb - np.log10(np.mean(ampratio))
mw_2016 = mw - np.log10(np.mean(sampratio)) / 1.5
print(mb_2016)

print(mw_2016)







